# Multiple chains

Runnables can easily be used to string together multiple Chains

In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
model_config_path = os.path.abspath(os.path.join('../custom_llms/'))
sys.path.insert(0, module_path)
sys.path.insert(0, model_config_path)

from custom_llms import (
    ZhipuAIEmbeddings,
    Zhipuai_LLM,
    load_api
)
api_key = load_api()
model = Zhipuai_LLM(zhipuai_api_key=api_key)

In [2]:
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "spanish"})

'El municipio de Obama proviene de la ciudad de Honolulu, que se encuentra ubicada en Hawaii, Estados Unidos. Nació allí el 4 de agosto de 1961. Sus padres se conocieron y casaron en Honolulu, y pasó gran parte de su infancia allí antes de mudarse a Indonesia y luego regresar a los Estados Unidos para realizar sus estudios superiores.'

In [3]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

In [4]:
question_generator.invoke("warm")

ChatPromptValue(messages=[HumanMessage(content='What is the color of Peach and the flag of The country with a flag containing a peachy pink color is Libya.?')])

In [5]:
prompt = question_generator.invoke("warm")
model.invoke(prompt)

"The color of coral fruit refers to the vibrant orange-pink hue of the fruit itself. The Republic of Maldives' flag features three horizontal bands of color: red, representing the Maldives' revolution; green, symbolizing Islam; and white, standing for peace and unity. While coral is not explicitly represented in the flag, it is often associated with the Maldives' beautiful coral reefs, which contribute to the country's renowned underwater beauty."

## Branching and Merging
You may want the output of one component to be processed by 2 or more other components. RunnableParallels let you split or fork the chain so multiple components can process the input in parallel. Later, other components can join or merge the results to synthesize a final response. This type of chain creates a computation graph that looks like the following:

     Input
      / \
     /   \
 Branch1 Branch2
     \   /
      \ /
      Combine

In [6]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | model
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | model
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | model
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | model
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [7]:
chain.invoke({"input": "scrum"})

"System: Title: A Balanced Perspective on Implementing Scrum: A Critique and Response\\\\n\\\\nAfter carefully considering the pros and cons of implementing Scrum, it becomes evident that while the methodology offers numerous benefits such as increased collaboration, task prioritization, and faster product delivery, it also has its share of drawbacks like inflexibility, overreliance on ceremonies, limited scalability, and potential mismanagement. To ensure a successful implementation and improved project management, organizations should weigh the advantages and disadvantages and adapt the Scrum framework accordingly.\\\\n\\\\nResponse:\\\\n\\\\n1. Flexibility and adaptability: To address the issue of inflexibility in the face of changing requirements, organizations can adopt a hybrid approach that combines Scrum with other methodologies like Kanban or DevOps. This enables teams to respond quickly to changing business requirements while still benefiting from the structure and predictabi